In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (15,10)

# Input

In [2]:
# from google.colab import files
# train = files.upload()
# test = files.upload()

In [3]:
# import io
# train = pd.read_csv(io.BytesIO(train['train.csv']))
# test = pd.read_csv(io.BytesIO(test['test.csv']))

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
titanic = train.append(test, ignore_index=True)
train_idx = len(train)
test_idx = len(titanic) - len(test)

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [7]:
titanic['Embarked'].fillna(titanic['Embarked'].mode()[0], inplace=True)

## notna: titanic data without nulls, used for training
## isna: titanic data with nulls, used for target

In [8]:
notna = titanic[titanic['Cabin'].notna()]

In [9]:
notna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 1 to 1305
Data columns (total 12 columns):
Age            272 non-null float64
Cabin          295 non-null object
Embarked       295 non-null object
Fare           295 non-null float64
Name           295 non-null object
Parch          295 non-null int64
PassengerId    295 non-null int64
Pclass         295 non-null int64
Sex            295 non-null object
SibSp          295 non-null int64
Survived       204 non-null float64
Ticket         295 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 30.0+ KB


In [10]:
notna.drop(['Name', 'Ticket', 'Age'], inplace=True, axis=1)

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
deck = []

In [12]:
for x in notna['Cabin']:
    y = x.split(' ')
    deck.append(y[0][0:1])

In [13]:
notna['Deck'] = deck

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
isna = titanic[titanic['Cabin'].isna()]

In [15]:
isna.drop(['Age', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1, inplace=True)
notna.drop(['Cabin', 'Survived'], axis=1, inplace=True)

In [16]:
isna['Fare'].fillna(round(isna['Fare'].median()), inplace=True)

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
isna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1308
Data columns (total 7 columns):
Embarked       1014 non-null object
Fare           1014 non-null float64
Parch          1014 non-null int64
PassengerId    1014 non-null int64
Pclass         1014 non-null int64
Sex            1014 non-null object
SibSp          1014 non-null int64
dtypes: float64(1), int64(4), object(2)
memory usage: 63.4+ KB


In [18]:
notna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 1 to 1305
Data columns (total 8 columns):
Embarked       295 non-null object
Fare           295 non-null float64
Parch          295 non-null int64
PassengerId    295 non-null int64
Pclass         295 non-null int64
Sex            295 non-null object
SibSp          295 non-null int64
Deck           295 non-null object
dtypes: float64(1), int64(4), object(3)
memory usage: 20.7+ KB


# Predicting the 'Deck' column


### training data -> notna
### target data -> isna

In [19]:
X = pd.get_dummies(notna.iloc[:, :-1])
y = notna['Deck']
target = pd.get_dummies(isna)

In [20]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
tree = DecisionTreeClassifier()

In [22]:
tree.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
pred = []

In [24]:
pred = pred + list(tree.predict(target[0:295]))

In [25]:
pred = pred + list(tree.predict(target[295:590]))

In [26]:
pred = pred + list(tree.predict(target[590:885]))

In [27]:
pred = pred + list(tree.predict(target[885:]))

In [28]:
isna['Deck'] = pred

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
isna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1308
Data columns (total 8 columns):
Embarked       1014 non-null object
Fare           1014 non-null float64
Parch          1014 non-null int64
PassengerId    1014 non-null int64
Pclass         1014 non-null int64
Sex            1014 non-null object
SibSp          1014 non-null int64
Deck           1014 non-null object
dtypes: float64(1), int64(4), object(3)
memory usage: 71.3+ KB


In [30]:
notna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 1 to 1305
Data columns (total 8 columns):
Embarked       295 non-null object
Fare           295 non-null float64
Parch          295 non-null int64
PassengerId    295 non-null int64
Pclass         295 non-null int64
Sex            295 non-null object
SibSp          295 non-null int64
Deck           295 non-null object
dtypes: float64(1), int64(4), object(3)
memory usage: 20.7+ KB


### Fitting the 'Deck' column

In [31]:
temp = [None]*1309
titanic['Deck'] = temp

In [32]:
passid_1 = list(notna['PassengerId'])

In [33]:
for x in list(titanic[titanic['PassengerId'].isin(passid_1)].index):
    titanic['Deck'][x] = notna['Deck'][x]

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
passid_2 = list(isna['PassengerId'])

In [35]:
for x in list(titanic[titanic['PassengerId'].isin(passid_2)].index):
    titanic['Deck'][x] = isna['Deck'][x]

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
titanic.drop('Cabin',inplace=True, axis=1)

In [37]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Embarked       1309 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Deck           1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [38]:
titanic['Fare'].fillna(titanic['Fare'].median(), inplace=True)

In [39]:
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch']

In [40]:
titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Predicting the 'Age' column

In [41]:
titanic.groupby(['Sex', 'Pclass']).median()['Age']

Sex     Pclass
female  1         36.0
        2         28.0
        3         22.0
male    1         42.0
        2         29.5
        3         25.0
Name: Age, dtype: float64

In [42]:
indices = list(titanic[titanic['Age'].isna()].index)

In [43]:
len(indices)

263

In [44]:
indices_m = []
indices_f = []

In [45]:
for x in indices:
  if titanic['Sex'][x] == 'male':
    indices_m.append(x)
  else:
    indices_f.append(x)

In [46]:
for x in indices_m:
  if titanic['Pclass'][x] == 1:
    titanic['Age'][x] = 36.0
  elif titanic['Pclass'][x] == 2:
    titanic['Age'][x] = 28.0
  else:
    titanic['Age'][x] = 22.0

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
for x in indices_f:
  if titanic['Pclass'][x] == 1:
    titanic['Age'][x] = 42.0
  elif titanic['Pclass'][x] == 2:
    titanic['Age'][x] = 29.0
  else:
    titanic['Age'][x] = 25.0

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [48]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
Age            1309 non-null float64
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
Survived       891 non-null float64
Ticket         1309 non-null object
Deck           1309 non-null object
FamilySize     1309 non-null int64
dtypes: float64(3), int64(3), object(5)
memory usage: 112.6+ KB


## Extracting name title

In [49]:
title = []
for x in titanic['Name']:
    title.append(x.split(',')[1].split('.')[0].strip())

In [50]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

In [51]:
titanic['Title'] = title

In [52]:
titanic['Title'] = titanic['Title'].map(normalized_titles)

In [53]:
titanic['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Officer'], dtype=object)

## Removing unnecessary features

In [54]:
titanic.drop(['Name', 'PassengerId', 'Ticket'], inplace=True, axis=1)

In [55]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
Age           1309 non-null float64
Embarked      1309 non-null object
Fare          1309 non-null float64
Pclass        1309 non-null int64
Sex           1309 non-null object
Survived      891 non-null float64
Deck          1309 non-null object
FamilySize    1309 non-null int64
Title         1309 non-null object
dtypes: float64(3), int64(2), object(4)
memory usage: 92.1+ KB


# --------------------------------------------------------------------

# !!! Stop here and review code for more feature enineering !!!

# ---------------------------------------------------------------------

# Caution! Conversion of data into dummies and arrays below

In [56]:
dummies = pd.get_dummies(titanic)

In [57]:
dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 24 columns):
Age              1309 non-null float64
Fare             1309 non-null float64
Pclass           1309 non-null int64
Survived         891 non-null float64
FamilySize       1309 non-null int64
Embarked_C       1309 non-null uint8
Embarked_Q       1309 non-null uint8
Embarked_S       1309 non-null uint8
Sex_female       1309 non-null uint8
Sex_male         1309 non-null uint8
Deck_A           1309 non-null uint8
Deck_B           1309 non-null uint8
Deck_C           1309 non-null uint8
Deck_D           1309 non-null uint8
Deck_E           1309 non-null uint8
Deck_F           1309 non-null uint8
Deck_G           1309 non-null uint8
Deck_T           1309 non-null uint8
Title_Master     1309 non-null uint8
Title_Miss       1309 non-null uint8
Title_Mr         1309 non-null uint8
Title_Mrs        1309 non-null uint8
Title_Officer    1309 non-null uint8
Title_Royalty    1309 non-null uint8


In [58]:
train = dummies[:train_idx]
test = dummies[test_idx:]

In [59]:
train['Survived'] = train['Survived'].astype(int)

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
test.drop('Survived', axis=1, inplace=True)

C:\Users\Sudeep\Miniconda3\envs\tf\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Feature Scaling

In [61]:
from sklearn.preprocessing import StandardScaler

In [62]:
ss = StandardScaler()

In [63]:
X = ss.fit_transform(train.drop('Survived', axis=1).values)
y = train['Survived'].values
test = ss.fit_transform(test.values)

# Data available:
## train, test
## X, y (from train)

# ----------------------------------

# Machine Learning starts here

# ----------------------------------

## Using Random Forest

In [ ]:
param_dict = {
    'max_depth': [n for n in range(9, 14)],     
    'min_samples_split': [n for n in range(4, 11)], 
    'min_samples_leaf': [n for n in range(2, 5)],     
    'n_estimators': [n for n in range(10, 60, 10)] }         

In [ ]:
X = train.drop('Survived', axis=1)
y = train['Survived']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=10, random_state=None)
forest = RandomForestClassifier()
grid = GridSearchCV(estimator=forest, param_grid=param_dict, cv=10, n_jobs=-1)

In [ ]:
'''grid search on random forest'''

grid.fit(X,y)
grid.best_params_

In [ ]:
forest = RandomForestClassifier(max_depth=12, min_samples_leaf=3, min_samples_split=6, n_estimators=30)

In [ ]:
'''random forest'''
'''using stratified k fold cross val'''

for train_index, test_index in skf.split(X,y): 
    print("Train:", train_index, "Test:", test_index) 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    forest.fit(X_train, y_train)
    pred = forest.predict(test)

In [ ]:
kaggle = pd.DataFrame({'PassengerId': pd.DataFrame(pd.read_csv('test.csv'))['PassengerId'], 'Survived': pred})
kaggle.to_csv('submission_2.csv', index=False)

## Using SVM

In [103]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [104]:
clf = svm.SVC(kernel='linear')
skf = StratifiedKFold(n_splits=10)

In [105]:
parameters = {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
              'gamma': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}

In [106]:
grid = GridSearchCV(clf, parameters, n_jobs=-1, cv=10)

In [107]:
grid.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'gamma': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [108]:
grid.best_params_

{'C': 0.1, 'gamma': 0.001}

In [109]:
clf = svm.SVC(kernel='linear', C=0.1, gamma=0.001)

In [110]:
for train_index, test_index in skf.split(X,y): 
    print("Train:", train_index, "Test:", test_index) 
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    clf.fit(X_train, y_train)
    pred = clf.predict(test)

Train: [ 82  84  85  88  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269
 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287
 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305
 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323
 324 325 326 327 328 329 330 331 332 333 334

In [111]:
kaggle = pd.DataFrame({'PassengerId': pd.DataFrame(pd.read_csv('test.csv'))['PassengerId'], 'Survived': pred})
kaggle.to_csv('submission_4.csv', index=False)